## Set up environment

The cell below will install the python libraries necessary to run this notebook

In [ ]:
! pip install anndata matplotlib numpy pandas scipy umap-learn

After installing the libraries, you need to restart the kernel so that you can import the libraries into this notebook.

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import anndata
import datetime
import hashlib
import json
import numpy as np
import pandas as pd
import pathlib
import scipy.sparse
import subprocess
import tempfile
import umap

Let's define (and create, if necessary) a directory in which we can store all of the data generated and consumed by this notebook.

In [ ]:
parent_data_dir = pathlib.Path('data')
if not parent_data_dir.exists():
    parent_data_dir.mkdir()

assert parent_data_dir.is_dir()

## Downloading example data and writing it to an h5ad file

The cell below downloads the data from the NEMO archive (if necessary). It will also validate that the file has the expected md5 checksum. The data in question is taken from Langlieb et al. 2023 (https://doi.org/10.1101/2023.03.06.531307)

In [ ]:
data_url = "https://data.nemoarchive.org/biccn/grant/u19_huang/macosko_regev/transcriptome/sncell/10X_v2/mouse/processed/counts/pBICCNsMMrMOPi70470511Bd180328.mex.tar.gz"
data_dst = parent_data_dir / "downloaded_file.tar.gz"

if not data_dst.exists():
    args = [
        "curl",
        "-o",
        str(data_dst.resolve().absolute()),
        data_url
    ]

    process = subprocess.Popen(args)
    process.wait()

assert data_dst.is_file()

expected_hash = "81ffc75a15bc40e32888bb2db9eecb52"
hasher = hashlib.md5()
with open(data_dst, "rb") as src:
    hasher.update(src.read())

assert hasher.hexdigest() == expected_hash
print("=======FILE SUCCESSFULLLY DOWNLOADED=======")

The cell below will untar the downloaded file.

In [ ]:
data_dir = parent_data_dir / 'untarred_data'
if not data_dir.exists():
    data_dir.mkdir()

args = [
    "tar",
    "-xvf",
    str(data_dst.resolve().absolute()),
    "--directory",
    str(data_dir.resolve().absolute())
]
process = subprocess.Popen(args)
process.wait()
print(f"=======FILE SUCCESSFULLY UNTARRED TO {data_dir}=======")

The cell below will write the data into an h5ad file that is suited for submission to MapMyCells.

In [ ]:
min_genes = 100  # only use wells with this many non-zero gene counts (or more)

h5ad_path = parent_data_dir / f"example_h5ad_min_genes_{min_genes:04}.h5ad"

if h5ad_path.exists():
    h5ad_path.unlink()

actual_data_dir = [n for n in data_dir.iterdir() if n.is_dir()][0]

gene_file_path = actual_data_dir / 'genes.tsv'
assert gene_file_path.is_file()

# Assemble a list of gene identifiers to serve as the index of 'var' in the h5ad file.
# We are going to make things more difficult on ourselves, identifying the genes
# by their symbols, rather than their EnsemblIDs (which are available in the .tsv
# file). MapMyCells prefers to work in EnsemblID, but can map common gene symbols
# to EnsemblID. We are using the gene symbols here to showcase that functionality.
#
# In "actual" production, it is best practice to identify all genes using
# EnsemblID.

gene_symbols = []
gene_identifiers = []
symbol_to_ct = dict()

with open(gene_file_path, 'r') as src:
    for line in src:
        line = line.split()
        identifier = line[0]
        symbol = line[1]
        gene_identifiers.append(identifier)
        gene_symbols.append(symbol)
        if symbol not in symbol_to_ct:
            symbol_to_ct[symbol] = 0
        symbol_to_ct[symbol] += 1

gene_list = []
for symbol, identifier in zip(gene_symbols, gene_identifiers):
    if symbol_to_ct[symbol] == 1:
        gene_list.append(symbol)
    else:
        gene_list.append(identifier)

assert len(set(gene_list)) == len(gene_list)

print(f"=======READ GENES FROM {gene_file_path}=======")

# Assemble a list of well barcodes to serve as the index of obs in our
# h5ad file.

barcode_file_path = actual_data_dir / 'barcodes.tsv'
assert barcode_file_path.is_file()
with open(barcode_file_path, 'r') as src:
    cell_list = [c.strip() for c in src]
print(f"=======READ CELL BARCODES FROM {barcode_file_path}=======")

# Identify the wells that pass our cut on min_genes.

matrix_file_path = actual_data_dir / 'matrix.mtx'

with open(matrix_file_path, 'r') as src:
    src.readline()
    src.readline()
    src.readline()
    cell_idx = np.array(
        [int(r.strip().split()[1])-1 for r in src])

unq_idx, ct = np.unique(cell_idx, return_counts=True)
valid_cell_idx = np.sort(unq_idx[(ct>=min_genes)])
cell_list = [cell_list[ii] for ii in valid_cell_idx]


# dict mapping old cell idx to new cell idx
cell_idx_transform = {old:new for new, old in enumerate(valid_cell_idx)}

valid_cells = set(valid_cell_idx)

print(f"======={len(valid_cells)} CELLS PASSED gene ct>={min_genes}=======")

obs = pd.DataFrame([{'barcode': c} for c in cell_list]).set_index('barcode')
var = pd.DataFrame([{'gene_id': g} for g in gene_list]).set_index('gene_id')

# Put together the cell-by-gene matrix

row_idx = []
col_idx = []
counts = []
with open(matrix_file_path, 'r') as src:
    src.readline()
    src.readline()
    src.readline()
    for line in src:
        params = line.strip().split()
        this_gene = int(params[0])-1
        this_cell = int(params[1])-1
        this_ct = int(params[2])
        if this_cell in valid_cells:
            row_idx.append(cell_idx_transform[this_cell])
            col_idx.append(this_gene)
            counts.append(this_ct)

print(f"=======READ MATRIX FROM {matrix_file_path}=======")

X_data = scipy.sparse.csr_matrix(
    (
     counts,
     (row_idx, col_idx)
    ),
    shape=(len(obs), len(var))
)

a_data = anndata.AnnData(
    X=X_data,
    obs=obs,
    var=var
)

a_data.write_h5ad(h5ad_path, compression='gzip', compression_opts=4)

print(f"\n=======WROTE h5ad FILE TO {h5ad_path.resolve().absolute()}=======")

timestamp = datetime.datetime.now().isoformat().split('T')[0]
salt = 0
mapping_dir = None
while mapping_dir is None or mapping_dir.exists():
    mapping_dir_name = f"mapping_{timestamp}_{salt:03}"
    mapping_dir = parent_data_dir / mapping_dir_name
    if mapping_dir.exists():
        contents = [n for n in mapping_dir.iterdir()]
        if len(contents) == 0:
            mapping_dir.rmdir()
    salt += 1

mapping_dir.mkdir(exist_ok=False)

print(
f"=======PLEASE SUBMIT h5ad FILE TO MapMyCells AT=======\n"
"https://knowledge.brain-map.org/mapmycells/process/\n"
f"=======AND DOWNLOAD RESULTS TO {mapping_dir.resolve().absolute()}======="
)


Read the original h5ad file so that we can inspect its contents.

In [ ]:
raw_data = anndata.read_h5ad(h5ad_path, backed='r')

Note that the `obs` object of the unlabeled h5ad file is indexed on the unique identifiers of each cell.

In [ ]:
raw_data.obs

The `var` object is indexed on the unique identifiers of each gene.

**Note:** MapMyCells prefers to work in EnsemblID. However, it does contain code to translate from common gene symbols to EnsemblIDs if you cannot directly provide the EnsemblIDs yourself.

In [ ]:
raw_data.var

The `X` element contains the **raw** (i.e. not the log-normalized) counts of each gene measured in each cell.

In [ ]:
raw_data.X

## Unzip and inspect the files produced by MapMyCells

**Note:** If you have not already, submit the h5ad file created above (see `h5ad_path`) for processing at

https://knowledge.brain-map.org/mapmycells/process/

and then download the results to `mapping_dir`.

### Inspect MapMyCells results

MapMyCells returns all of its outputs in a single .zip file. To usefully analyze it, you must unzip the file into its constituent parts.

In [ ]:
zip_file_path = [n for n in mapping_dir.rglob('**/*.zip')]
if len(zip_file_path) != 1:
    raise RuntimeError(
        f"Found {len(zip_file_path)} zip files in {mapping_dir}; expected exactly 1"
    )
zip_file_path = zip_file_path[0]
zip_file_stem = zip_file_path.stem
args = [
    "tar",
    "-xvf",
    str(zip_file_path.resolve().absolute()),
    "--directory",
    str(mapping_dir.resolve().absolute())
]

process = subprocess.Popen(args)
process.wait()

# the path to the CSV file containingin mapping results
csv_path = mapping_dir / f"{zip_file_stem}.csv"
assert csv_path.is_file()

# the path to the JSON file containing mapping results
json_path = mapping_dir / f"{zip_file_stem}.json"
assert json_path.is_file()


### CSV mapping results

Let us now look at the contents of the CSV file returned by MapMyCells.

The first 3-4 (depending on the chosen taxonomy and algorithm) lines contain metadata describing the MapMyCells run. These are flagged with a `#`.

In [ ]:
with open(csv_path, 'r') as src:
    for line in src.readlines()[:6]:
        print(line)

You can read the CSV file in with `pandas`. Be sure to tell pandas to ignore lines that begin with a `#`.

In [ ]:
csv_results = pd.read_csv(csv_path, comment='#')
csv_results

Each row in the resulting DataFrame corresponds to a cell. The columns list the cell's ID, the machine- and human-readable names of the cell types teh cell was assigned to at each level, as well as the `bootstrapping_probability` for the assignment.

`bootstrapping_probability` is a quality metric for the mapping. Recall that hierarchical mapping maps the cell 100 times, using a random subset of marker genes for each iteration, and assigning the cell to the cell type that is most correlated with the cell in the plurality of those 100 iterations. `bootstrapping_probability` is the fraction of the 100 iterations that chose the final assignment.

In [ ]:
list(csv_results.columns)

### JSON mapping results

Now let's read in the JSON file. This contains the same results as the CSV file, along with additional quality metrics and metadata.

In [ ]:
with open(json_path, 'rb') as src:
    json_results = json.load(src)
json_results.keys()

`json_results` is a dict. The mapping results are stored in `results`, which maps to a list of dicts, each of which represents a cell from the unlabeled dataset.

The dict representing each cell is keyed on the machine-readable labels for levels in the cell type taxonomy (in our case `CCN20230722_CLUS`, `CCN20230722_SUBC`, `CCN20230722_SUPT`, `CCN20230722_CLAS`). Under each of these keys you will find the same data as in the CSV file. Additionally, you will find:

- `avg_correlation`, the correlation coefficient between the cell and the chosen cell type *averaged over the iterations that selected that cell type.*
- `runner_up_assignment`, `runner_up_correlation`, `runner_up_probability`, lists indicating the labels, `avg_correlation`, and `bootstrapping_probability` for the five cell types that got the next highest number of "votes" among the 100 mapping iterations.
- `aggregate_probability`, the running product of `bootstrapping_probability` from most gross to most fine level in the taxonomy.
- `directly_assigned` a boolean indicating whether or not the class was chosen directly (in the Whole Mouse Brain taxonomy, supertype is not directly assigned; the assignment goes directly from subclass to cluster and supertype is inferred from the chosen cluster)

In [ ]:
print(json.dumps(json_results['results'][1000], indent=2, sort_keys=True))

The `gene_identifier_mapping` key maps to dict showing how MapMyCells mapped your gene symbols to EnsemblID. If MapMyCells could not find a gene in its lookup tables, the gene will be mapped to a nonsense identifier beginning with `unmapped_`

In [ ]:
json_results['gene_identifier_mapping']

Information about the cell type taxonomy to which your data was mapped is stored under the `taxonomy_tree` key.

In [ ]:
taxonomy_tree = json_results['taxonomy_tree']

In [ ]:
taxonomy_tree.keys()

`hierarchy` lists the levels of the cell type taxonomy from most gross to most fine.

In [ ]:
taxonomy_tree['hierarchy']

The machine-readable labels can be translated to human-readable labels using the `hierarchy_mapper`.

In [ ]:
taxonomy_tree['hierarchy_mapper']

Parent-child relationships are listed under the various levels in the taxonomy. For instance, `CCN20230722_CLAS` keys to a dict which maps the individual cell types (e.g. `CS20230722_CLAS_01`) to lists of their children.

In [ ]:
taxonomy_tree['CCN20230722_CLAS']

The `name_mapper` field provides for a translation between machine-readable labels and human-readable names.

In [ ]:
taxonomy_tree['name_mapper'][taxonomy_tree['hierarchy'][0]]

## QC plot

Now let's do a quick QC assessment of our mapping. Let's plot `avg_correlation` and `bootstrapping_probability` for a cell's class assignment (the highest level in the Whole Mouse Brain taxonomy) against the number of genes measured with non-zero counts in that cell.

In [ ]:
level = 'CCN20230722_CLAS'

cell_barcodes = raw_data.obs.index.values

# convert cell type mapping into a dict keyed on each cell's barcode
mapping_result = {c['cell_id']: c for c in json_results['results']}

# get an array of how many genes were measured with more than 0 counts in each cell
n_genes = (raw_data.X[()].toarray()>0).sum(axis=1)

# assemble arrays of the two QC metrics
avg_corr = np.array([mapping_result[b][level]['avg_correlation'] for b in cell_barcodes])
bootstrapping_prob = np.array([mapping_result[b][level]['bootstrapping_probability'] for b in cell_barcodes])

fig = plt.figure(figsize=(15,5))
corr_axis = fig.add_subplot(1,3,1)
prob_axis = fig.add_subplot(1,3,2)
corr_v_prob_axis = fig.add_subplot(1,3,3)

level_name = taxonomy_tree['hierarchy_mapper'][level]

corr_axis.set_title(level_name)
corr_axis.scatter(n_genes, avg_corr, s=1)
corr_axis.set_xlabel('N non-zero genes')
corr_axis.set_ylabel('average correlation')
corr_axis.set_xscale('log')

prob_axis.scatter(n_genes, bootstrapping_prob, s=1)
prob_axis.set_xlabel('N non-zero genes')
prob_axis.set_ylabel('bootstrapping probability')
prob_axis.set_xscale('log')

corr_v_prob_axis.scatter(avg_corr, bootstrapping_prob, s=1)
corr_v_prob_axis.set_xlabel('average correlation')
corr_v_prob_axis.set_ylabel('bootstrapping probability')

## UMAP plot

Below we demonstrate how to generate a UMAP of the unlabeled data and plot it, coloring the points according to the cell types assigned by MapMyCells.

First, define a helper function to generate and plot the embedding.

In [ ]:
def plot_embedding(
        barcode_to_label,
        label_order,
        fontsize=15):
    """
    Generate and plot a UMAP embedding of the raw data.

    Parameters
    ----------
    barcode_to_label:
        A dict mapping cell barcodes to labels (only includes
        cells that we want included in the embedding)
    label_order:
        order in which we want labels to appear on the color map.
    fontsize:
        size of font to be used in legend

    Returns
    -------
    None
        generates and displays the plot of the embedding
    """

    level = 'CCN20230722_CLAS'
    reducer = umap.UMAP()

    _barcodes = raw_data.obs.index.values
    data = raw_data.X[()].toarray()

    filter_mask = np.zeros(data.shape[0], dtype=bool)
    for ii, b in enumerate(_barcodes):
        if b in barcode_to_label:
            filter_mask[ii] = True

    if not hasattr(plot_embedding, 'filter_mask'):
        plot_embedding.embedding =  None

    run_embedding = False
    if plot_embedding.embedding is None:
        run_embedding = True
    elif not np.array_equal(filter_mask, plot_embedding.filter_mask):
        run_embedding = True

    if run_embedding:
        plot_embedding.embedding = reducer.fit_transform(data[filter_mask, :])
        plot_embedding.filter_mask = filter_mask

    embedding = plot_embedding.embedding

    label_to_idx = {label:ii for ii, label in enumerate(label_order)}
    
    assignments = np.array(
        [
            label_to_idx[
                barcode_to_label[b]
            ] for b in _barcodes[filter_mask]
        ]
    )

    ticks = [ii for ii in range(len(label_order))]

    fig = plt.figure(figsize=(10,10))
    axis = fig.add_subplot(1,1,1)
    for label in label_order:
        cell_mask = (assignments==label_to_idx[label])
        axis.scatter(embedding[cell_mask, 0], embedding[cell_mask, 1], label=label, s=10)
    axis.legend(loc=0, fontsize=fontsize)

Let's just color the points by their class assigment, assigning classes with `Glut` in the name to one color, `GABA` to another, and all other classes their own color.

In [ ]:
level = 'CCN20230722_CLAS'
corr_cut = 0.4

node_to_label = dict()
for node in taxonomy_tree[level]:
    name = taxonomy_tree['name_mapper'][level][node]['name']
    if 'Glut' in name:
        label = 'Glut'
    elif 'GABA' in name:
        label = 'GABA'
    else:
        label = name
    node_to_label[node] = label

barcode_to_label = dict()
for barcode in mapping_result:
    cell = mapping_result[barcode]
    if cell[level]['avg_correlation'] < corr_cut:
        continue
    barcode_to_label[barcode] = node_to_label[cell[level]['assignment']]

label_order = ['Glut', 'GABA']
for label in set(barcode_to_label.values()):
    if label not in label_order:
        label_order.append(label)

In [ ]:
label_order

In [ ]:
%%time
plot_embedding(barcode_to_label=barcode_to_label, label_order=label_order)

Now let's look at the same plot, coloring each individual `Glut` class separately.

In [ ]:
node_to_label = dict()
for node in taxonomy_tree[level]:
    name = taxonomy_tree['name_mapper'][level][node]['name']
    if 'Glut' in name:
        idx = 0
        label = name
    else:
        label = 'Other'
    node_to_label[node] = label

barcode_to_label = dict()
for barcode in mapping_result:
    cell = mapping_result[barcode]
    if cell[level]['avg_correlation'] < corr_cut:
        continue
    barcode_to_label[barcode] = node_to_label[cell[level]['assignment']]

label_order = ['Other']
for label in set(barcode_to_label.values()):
    if label not in label_order:
        label_order.append(label)

In [ ]:
%%time
plot_embedding(barcode_to_label=barcode_to_label, label_order=label_order)

Look at the individual `GABA` classes.

In [ ]:
node_to_label = dict()
for node in taxonomy_tree[level]:
    name = taxonomy_tree['name_mapper'][level][node]['name']
    if 'GABA' in name:
        idx = 0
        label = name
    else:
        label = 'Other'
    node_to_label[node] = label

barcode_to_label = dict()
for barcode in mapping_result:
    cell = mapping_result[barcode]
    if cell[level]['avg_correlation'] < corr_cut:
        continue
    barcode_to_label[barcode] = node_to_label[cell[level]['assignment']]

label_order = ['Other']
for label in set(barcode_to_label.values()):
    if label not in label_order:
        label_order.append(label)

In [ ]:
%%time
plot_embedding(barcode_to_label=barcode_to_label, label_order=label_order)

Now let's color by subclass, but only consider children of the `Glut` class `01 IT-ET Glut` (`CS20230722_CLAS_01`)

In [ ]:
subclass_level = 'CCN20230722_SUBC'
class_level = 'CCN20230722_CLAS'
chosen_class = 'CS20230722_CLAS_01'
node_to_label = dict()
corr_cut = 0.4

for node in taxonomy_tree[class_level][chosen_class]:
    name = taxonomy_tree['name_mapper'][subclass_level][node]['name']
    node_to_label[node] = name

barcode_to_label = dict()
for barcode in mapping_result:
    cell = mapping_result[barcode]
    if cell[class_level]['avg_correlation'] < corr_cut:
        continue
    if cell[subclass_level]['avg_correlation'] < corr_cut:
        continue
    if cell[class_level]['assignment'] != chosen_class:
        continue
    barcode_to_label[barcode] = node_to_label[cell[subclass_level]['assignment']]

label_order =list(set(barcode_to_label.values()))
label_order.sort()

In [ ]:
len(barcode_to_label)

In [ ]:
%%time
plot_embedding(barcode_to_label=barcode_to_label, label_order=label_order, fontsize=12)